In [1]:
import numpy as np
import pandas as pd

In [2]:
diamonds = pd.read_csv('./data/diamonds_train.csv.zip')
diamonds_predict = pd.read_csv('./data/diamonds_predict.csv')

## ML preprocessing

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [4]:
NUM_FEATS = ['carat', 'depth', 'table', 'x', 'y']
CAT_FEATS = ['cut', 'color', 'clarity']
FEATS = NUM_FEATS + CAT_FEATS
TARGET = 'price'

In [5]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
               ('scaler', StandardScaler())])

In [6]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
               ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [7]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                               ('cat', categorical_transformer, CAT_FEATS)])

In [8]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'table', 'x', 'y']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['cut', 'color', 'clarity'])])

In [9]:
pd.DataFrame(data=preprocessor.fit_transform(diamonds)).head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.867006,0.452019,0.247981,0.978807,0.921985,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-1.004557,0.871099,-0.199745,-1.226738,-1.179816,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.184434,2.617265,-1.095198,-0.097286,-0.176882,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.815298,1.429872,-0.647472,-0.933258,-0.883296,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.467458,-0.875068,0.695707,0.729794,0.677793,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Model

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
diamonds_train, diamonds_test = train_test_split(diamonds)

In [12]:
print(diamonds_train.shape)
print(diamonds_test.shape)

(30341, 10)
(10114, 10)


In [15]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

In [16]:
model = Pipeline(steps=[('preprocessor', preprocessor),
                       ('regressor', RandomForestRegressor())])

In [17]:
model.fit(diamonds_train[FEATS], diamonds_train[TARGET]);

## Model performance

In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [22]:
y_test = model.predict(diamonds_test[FEATS])
y_train = model.predict(diamonds_train[FEATS])

In [25]:
print(f"test mean sq error: {mean_squared_error(y_pred=y_test, y_true=diamonds_test[TARGET], squared=False)}")
print(f"train men sq error: {mean_squared_error(y_pred=y_train, y_true=diamonds_train[TARGET], squared=False)}")

test mean sq error: 548.1937284473945
train men sq error: 212.13981024366512


## R2 test

In [29]:
print(f"test mean sq error: {r2_score(y_pred=y_test, y_true=diamonds_test[TARGET])}")
print(f"train mean sq error: {r2_score(y_pred=y_train, y_true=diamonds_train[TARGET])}")

test mean sq error: 0.9809356198441229
train mean sq error: 0.997186868129244


Based on r2 test we are having overfitting, I'll check doing the
grid search 

In [30]:
from sklearn.model_selection import cross_val_score


In [31]:
scores = cross_val_score(model,
                        diamonds[FEATS],
                        diamonds[TARGET],
                        scoring='neg_root_mean_squared_error',
                        cv=5,
                        n_jobs=-1)

In [32]:
np.mean(-scores)

562.61031504157

In [33]:
from sklearn.model_selection import RandomizedSearchCV

In [34]:
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'regressor__n_estimators': [16, 32, 64, 128, 256, 512],
    'regressor__max_depth': [2, 4, 8, 16],
}

grid_search = RandomizedSearchCV(model,
                                param_grid,
                                verbose=10,
                                scoring='neg_root_mean_squared_error',
                                n_jobs=-1,
                                n_iter=32)
grid_search.fit(diamonds[FEATS], diamonds[TARGET])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed: 10.1min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['carat',
                                                                                'depth',
                                                                                'table',
                                                                                'x',
                                

In [35]:
grid_search.best_params_

{'regressor__n_estimators': 128,
 'regressor__max_depth': 16,
 'preprocessor__num__imputer__strategy': 'median'}

In [36]:
grid_search.best_score_

-558.2233877001138

## Submission

In [37]:
y_pred = grid_search.predict(diamonds_predict[FEATS])

In [39]:
submission_df = pd.DataFrame({'id': diamonds_predict['id'],
                             'price': y_pred})

In [40]:
submission_df.head()

,id,price
0,0,2981.492963
1,1,5395.236261
2,2,9622.977527
3,3,4122.457712
4,4,1693.166991


In [42]:
submission_df.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3953.317568
std,3892.928525,3947.109930
min,0.000000,370.635066
25%,3371.000000,947.508054
50%,6742.000000,2464.261515
75%,10113.000000,5329.280526
max,13484.000000,18087.372791


In [43]:
submission_df.price.clip(370, 20000, inplace=True)

In [46]:
submission_df.to_csv('./submissions/submission_08_spt.csv', index=False)